In [5]:
import pandas as pd
import nltk
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
stop = stopwords.words('english')

df1 = pd.read_csv("reviews_data_giant.csv")
df2 = pd.read_csv("reviews_data_more.csv")
df1 = df1.drop(['Unnamed: 0'],axis=1)
df1 = df1.drop(['Date'],axis=1)
df1 = df1.drop(['Rating'],axis=1)
df2 = df2.drop(['Unnamed: 0'],axis=1)
df2 = df2.drop(['Date'],axis=1)
df2 = df2.drop(['Rating'],axis=1)
full_df = pd.concat([df1, df2])
full_df

FileNotFoundError: [Errno 2] No such file or directory: 'reviews_data_giant.csv'

In [4]:
#get rid of all duplicates 
full_df = full_df.drop_duplicates(keep = 'first')
full_df = full_df.reset_index()
full_df = full_df.drop(['index'], axis=1)
full_df

NameError: name 'full_df' is not defined

In [ ]:
# Clean Reviews 
full_df['Review'] = full_df['Review'].str.replace(';', '\'')
full_df['Review'] = full_df['Review'].str.replace('<br&gt;', '')
full_df['Review'] = full_df['Review'].str.replace('&', '')
full_df['Review'] = full_df['Review'].str.replace('amp;', '')
full_df['Review'] = full_df['Review'].str.replace('amp', '')
full_df['Review'] = full_df['Review'].str.replace('  ', ' ')
full_df['Review'] = full_df['Review'].str.replace('#39\'', ' ')

# Make all lowercase
full_df['Review'] = full_df['Review'].apply(str.lower)

# Add custom stop words
custom_stop_words = ['food', 'burlington', 'would', '-']
stop = stop + custom_stop_words

In [ ]:
# Filter out stop words store in new collumn
full_df['Review'] = full_df['Review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
full_df['Review']

In [2]:
#put all the reviews in one 
#each element of all_reviews list is all of the reviews for a restaurant
all_reviews = []
restaur_reviews = ''
#start name
count = 0
new_name = full_df.loc[0, 'Name']
for i in range(0, len(full_df)):
    if str(full_df.loc[i, 'Name']) == str(new_name):
        string = str(full_df.loc[i, 'Review'])
        restaur_reviews += string + ' '
    else:
        #changing name
        all_reviews.append(restaur_reviews)
        restaur_reviews = ''
        new_name = full_df.loc[i, 'Name']
        #print(new_name)
        count += 1
        string = str(full_df.loc[i, 'Review'])
        restaur_reviews += string + ' '
#last review 
all_reviews.append(restaur_reviews)
        
print(count)

NameError: name 'full_df' is not defined

In [ ]:
#get rid of rows with about the same restaurant
full_df = full_df.drop(['Review'],axis=1)
full_df = full_df.drop_duplicates(keep = 'first')
full_df


In [ ]:
full_df = full_df.reset_index()
full_df = full_df.drop(['index'], axis=1)
full_df

In [ ]:
#add review text
new_df = pd.concat([full_df, pd.DataFrame(all_reviews, columns= ['Review text'])], axis=1)

In [ ]:
new_df

In [ ]:
#create a list of all the different food types, find which ones occur the most

all_food_types = []
for i in range(0,len(new_df)):
    temp_cats = new_df.loc[i, 'Category'].split(',')
    for word in temp_cats:
        #don't add the word if it is 'restaurant' or 'food'
        if word == 'restaurant' or word == 'restaurants' or word == 'food' or word == 'foo':
            continue
        else:
            #add each word to all food types
            all_food_types.append(word)

#list of top 40 categories
new_data = pd.Series(all_food_types).value_counts()[0:39].to_frame(name='counts')
top_categor = []
count = 0
#loop through rows
for index, row in new_data.iterrows():
    if count < 40:
        top_categor.append(index)
    count += 1



In [ ]:
#change category to a few main types of food
#create category 1 and category 2 with one word in each
for i in range(0, len(new_df)):
    #if we find one of the top 40 categories, make new column
    words_found = 0
    for word in top_categor:
        if new_df.loc[i, 'Category'].find(word) != -1:
            words_found += 1
            if words_found == 1:
                new_df.loc[i, 'Category 1'] = word
            elif words_found == 2:
                new_df.loc[i, 'Category 2'] = word
                
#now drop category
new_df = new_df.drop(['Category'], axis=1)

In [ ]:
new_df

In [ ]:
#clean the street names 
for i in range(0, len(new_df)):
    if str(new_df.loc[i, 'Street']).find(',') != -1:
        split = new_df.loc[i, 'Street'].split(',')
        new_df.loc[i, 'Street'] = split[0]
    elif new_df.loc[i, 'Street'] == 'Photo' or new_df.loc[i, 'Street'] == 'Phot'or new_df.loc[i, 'Street'] == 'Pho' or new_df.loc[i, 'Street'] == 'Tru' or new_df.loc[i, 'Street'] == 'Bar':
        new_df.loc[i, 'Street'] = 'NaN'
    elif new_df.loc[i, 'Street'] == 'St Paul St':
        new_df.loc[i, 'Street'] = 'Saint Paul St'
    new_df.replace(' ', '-')
new_df

In [ ]:
#clean prices
for i in range(0, len(new_df)):
    if new_df.loc[i, 'Price Tags'] == 'ul':
        new_df.loc[i, 'Price Tags'] = 'NaN'


In [ ]:
#create column to classify each restaurant 
#cut off is between 4 and 4.5
new_df['Customer Satisfaction'] = new_df['Average Rating'].replace([2.5, 3.0, 3.5], 'Low')
new_df['Customer Satisfaction'] = new_df['Customer Satisfaction'].replace( [4.0, 4.5, 5], 'High')

In [ ]:
new_df

In [ ]:
new_df.to_csv('full_cleaned_data.csv')

In [ ]:
#df.loc[df['Name'] == df.loc[33,'Name']]


In [ ]:
#load in different (updated) data set

In [14]:
df = pd.read_csv("full_cleaned_data.csv")
df = df.drop(['Unnamed: 0'], axis=1)

#one more cleaning thing
for i in range(0, len(df)):
    if str(df.loc[i, 'Street']).find('-') != -1:
        df.loc[i, 'Street'] = df.loc[i, 'Street'].replace('-', '_')
df.to_csv('full_cleaned_data.csv')